#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 7, 18)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,TISCO,"1,000",88.50,1,CUT,SET50,2022-04-29


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
1,6,B,IVL,3000,43.50,1,13WL,SET50,2022-05-31
2,8,B,KTC,2100,52.75,1,52WL,SET50,2022-04-18
3,18,S,GLOBAL,4500,18.50,0,C18.40,SET50,2022-02-02
4,20,S,TISCO,1000,88.50,1,CUT,SET50,2022-04-29


In [6]:
name = 'IVL'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,47.19,47.81,46.50,17188344.97,47.21
std,2.20,2.25,2.20,9581496.58,2.18
min,43.50,44.00,42.25,4604963.00,43.50
25%,45.25,46.00,44.75,10260392.25,45.50
50%,47.50,48.00,46.75,15229270.00,47.00
75%,49.00,49.56,48.56,20946406.75,49.00
max,51.50,52.75,50.50,44756656.00,51.25


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,AP,10000,9.00,1,MinP,SET100,2022-05-10
1,5,B,BCPG,9000,10.00,1,52WL,SET100,2022-02-02
2,7,B,KKP,3000,60.00,1,52WL,SET100,2022-04-28
3,10,B,RBF,9000,12.00,1,52WL,SET100,2022-02-02
4,11,B,RCL,3000,30.75,0,RD30pct,SET100,2022-08-27
5,12,B,SINGER,3000,35.50,0,52WL,SET100,2022-02-02
6,16,S,BCH,3000,21.00,1,CP1S,SET100,2022-08-31


In [8]:
name = 'RBF'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,16.23,16.57,16.02,7117096.48,16.34
std,1.71,1.79,1.71,5204501.62,1.76
min,13.00,13.40,12.80,1057817.00,13.30
25%,15.10,15.30,14.88,3674047.00,15.20
50%,15.95,16.20,15.80,5738559.00,16.00
75%,16.80,17.12,16.52,8631773.50,16.82
max,22.60,23.20,22.50,27045436.00,22.60


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
1,3,B,ASK,3000,30.00,1,52WL,SET,2022-03-03
2,9,B,MCS,5000,10.60,1,52WL,SET,2022-09-03
3,13,B,SNC,9000,16.50,1,1L,SET,2022-03-02
4,14,B,SSP,10000,8.50,1,52WL,SET,2022-02-02
5,15,B,TMT,12000,7.90,1,RD20pct,SET,2022-08-27
6,17,S,DIF,10000,14.60,0,CUT,SET,2022-08-10
7,19,S,SIS,3600,28.75,1,C28.50,SET,2022-03-10


In [10]:
name = 'JASIF'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,1.200000e+02,120.00
mean,10.63,10.71,10.54,2.495170e+07,10.64
std,0.61,0.58,0.62,4.528143e+07,0.59
min,8.80,8.85,8.60,2.238519e+06,8.65
25%,10.60,10.70,10.50,8.082650e+06,10.60
50%,10.90,10.90,10.80,1.344292e+07,10.90
75%,11.00,11.00,10.90,2.448689e+07,11.00
max,11.30,11.30,11.10,3.909313e+08,11.20


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AIT,18000,4.20,0,52WL,SET,2022-02-02
1,2,B,AP,10000,9.00,1,MinP,SET100,2022-05-10
2,3,B,ASK,3000,30.00,1,52WL,SET,2022-03-03
3,4,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
4,5,B,BCPG,9000,10.00,1,52WL,SET100,2022-02-02
5,6,B,IVL,3000,43.50,1,13WL,SET50,2022-05-31
6,7,B,KKP,3000,60.00,1,52WL,SET100,2022-04-28
7,8,B,KTC,2100,52.75,1,52WL,SET50,2022-04-18
8,9,B,MCS,5000,10.60,1,52WL,SET,2022-09-03
9,10,B,RBF,9000,12.00,1,52WL,SET100,2022-02-02
